# Persona to GPT-4 Interface

This notebook allows you to input a persona and get responses from GPT-4 using the configuration from the OpenAI config file.

## Setup Requirements

1. **Install dependencies**: Make sure you have Poetry installed and run:
   ```bash
   poetry install
   ```

2. **Environment variables**: Set your OpenAI API key:
   ```bash
   export OPENAI_API_KEY="your-api-key-here"
   ```
   Or create a `.env` file in the project root with:
   ```
   OPENAI_API_KEY=your-api-key-here
   ```

3. **Run in Poetry environment**: Start Jupyter in the Poetry environment:
   ```bash
   poetry run jupyter notebook
   ```

## Dependencies Used

This notebook uses the following key dependencies from `pyproject.toml`:
- `openai` (^1.78.1) - OpenAI API client
- `python-dotenv` (^1.1.0) - Environment variable management
- `tenacity` (^9.1.2) - Retry logic for API calls
- `tqdm` (^4.66.0) - Progress bars
- `google-generativeai` (^0.8.5) - Also supports Gemini if needed

In [ ]:
# Check if we're running in the correct Poetry environment
import sys
import subprocess

try:
    # Check if we're in a Poetry environment
    result = subprocess.run(['poetry', 'env', 'info', '--path'], 
                          capture_output=True, text=True, check=True)
    poetry_env_path = result.stdout.strip()
    if poetry_env_path in sys.executable:
        print("✅ Running in Poetry environment")
    else:
        print("⚠️  Not running in Poetry environment. Please start with: poetry run jupyter notebook")
except:
    print("⚠️  Poetry not found or not in Poetry environment")

# Import required libraries
import os
import asyncio
import yaml
import sys
from pathlib import Path

# Add the text_simulation directory to the path to import the LLM helper
sys.path.append('text_simulation')
from llm_helper import LLMConfig, process_prompts_batch

# Load environment variables if .env file exists
try:
    from dotenv import load_dotenv
    load_dotenv()
    print("✅ dotenv loaded successfully")
except ImportError:
    print("⚠️  dotenv not installed. Make sure OPENAI_API_KEY is set in your environment.")

# Check if OpenAI API key is available
if os.getenv('OPENAI_API_KEY'):
    print("✅ OPENAI_API_KEY found in environment")
else:
    print("❌ OPENAI_API_KEY not found. Please set it in your environment or .env file")

In [ ]:
# Load configuration from OpenAI config file
config_path = "text_simulation/configs/openai_config.yaml"

with open(config_path, 'r') as file:
    config_data = yaml.safe_load(file)

print("Loaded configuration:")
print(f"Model: {config_data['model_name']}")
print(f"Temperature: {config_data['temperature']}")
print(f"Max tokens: {config_data['max_tokens']}")
print(f"System instruction: {config_data['system_instruction'][:100]}...")

In [ ]:
# Create LLM configuration
llm_config = LLMConfig(
    model_name=config_data['model_name'],
    temperature=config_data['temperature'],
    max_tokens=config_data['max_tokens'],
    system_instruction=config_data['system_instruction'],
    max_retries=config_data['max_retries'],
    max_concurrent_requests=1  # Set to 1 for notebook usage
)

print("LLM configuration created successfully!")

In [ ]:
def create_persona_prompt(persona_profile, new_question):
    """Create a formatted prompt with persona profile and new question."""
    prompt = f"""Persona Profile:
{persona_profile}

New Survey Question:
{new_question}"""
    return prompt

async def get_gpt_response(persona_profile, new_question):
    """Get GPT-4 response for a given persona and question."""
    prompt = create_persona_prompt(persona_profile, new_question)
    
    # Create a single prompt tuple
    prompts = [("persona_response", prompt)]
    
    # Process the prompt
    results = await process_prompts_batch(
        prompts, 
        llm_config, 
        provider="openai",
        desc="Getting GPT-4 response"
    )
    
    return results["persona_response"]

print("Helper functions defined!")

## Input Your Persona and Question

Modify the cells below to input your persona profile and the new survey question you want the persona to answer.

In [ ]:
# Define your persona profile
persona_profile = """
Age: 35
Gender: Female
Education: Bachelor's degree in Business Administration
Income: $75,000 annually
Location: Urban area, East Coast
Previous survey responses:
- When asked about work-life balance: "I prioritize family time and try to maintain clear boundaries between work and personal life."
- When asked about technology adoption: "I'm generally cautious about new technologies and prefer to wait until they're proven and widely adopted."
- When asked about spending habits: "I'm a careful budgeter who prefers to save money for future goals rather than spend on immediate gratification."
""".strip()

print("Persona profile set:")
print(persona_profile)

In [ ]:
# Define your new survey question
new_question = """
How do you feel about remote work opportunities? Please provide your opinion in 2-3 sentences, considering your work-life balance preferences and technology comfort level.
""".strip()

print("New survey question:")
print(new_question)

## Get GPT-4 Response

Run the cell below to get the GPT-4 response as the defined persona.

In [ ]:
# Get the GPT-4 response
response_data = await get_gpt_response(persona_profile, new_question)

# Display the results
if "error" in response_data and response_data["error"]:
    print("Error occurred:")
    print(response_data["error"])
else:
    print("GPT-4 Response as the persona:")
    print("=" * 50)
    print(response_data["response_text"])
    print("=" * 50)
    
    # Display usage statistics
    if "usage_details" in response_data:
        usage = response_data["usage_details"]
        print(f"\nToken usage:")
        print(f"- Prompt tokens: {usage.get('prompt_token_count', 'N/A')}")
        print(f"- Completion tokens: {usage.get('completion_token_count', 'N/A')}")
        print(f"- Total tokens: {usage.get('total_token_count', 'N/A')}")

## Interactive Mode

Use the cell below to interactively test different questions with the same persona.

In [ ]:
# Interactive testing - modify this question and re-run
interactive_question = "What is your opinion on investing in cryptocurrency?"

response_data = await get_gpt_response(persona_profile, interactive_question)

if "error" in response_data and response_data["error"]:
    print("Error occurred:")
    print(response_data["error"])
else:
    print(f"Question: {interactive_question}")
    print("\nPersona Response:")
    print("-" * 30)
    print(response_data["response_text"])
    print("-" * 30)

## Batch Testing

Test multiple questions at once with the same persona.

In [ ]:
# Define multiple questions to test
test_questions = [
    "How do you prefer to shop for groceries?",
    "What factors influence your choice of vacation destinations?",
    "How do you stay informed about current events?"
]

# Process all questions
print("Processing multiple questions...\n")

for i, question in enumerate(test_questions, 1):
    print(f"Question {i}: {question}")
    
    response_data = await get_gpt_response(persona_profile, question)
    
    if "error" in response_data and response_data["error"]:
        print(f"Error: {response_data['error']}")
    else:
        print(f"Response: {response_data['response_text']}")
    
    print("-" * 80)
    print()